In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from scipy.io import arff
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from hyperopt import tpe, hp, fmin


#Reading the Dataset
#Read the dataset into a Pandas Data Frame!


df = pd.read_csv("KaggleV2-May-2016.csv")

df




,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110522,2.572134e+12,5651768,F,2016-05-03T09:15:35Z,2016-06-07T00:00:00Z,56,MARIA ORTIZ,0,0,0,0,0,1,No
110523,3.596266e+12,5650093,F,2016-05-03T07:27:33Z,2016-06-07T00:00:00Z,51,MARIA ORTIZ,0,0,0,0,0,1,No
110524,1.557663e+13,5630692,F,2016-04-27T16:03:52Z,2016-06-07T00:00:00Z,21,MARIA ORTIZ,0,0,0,0,0,1,No
110525,9.213493e+13,5630323,F,2016-04-27T15:09:23Z,2016-06-07T00:00:00Z,38,MARIA ORTIZ,0,0,0,0,0,1,No


In [13]:
#Does the dataset include any missing values? If so, drop them!
#Hint:
#Pandas can do that with one line of code!
print("missing values")
print(df.isna().sum())




missing values
PatientId         0
AppointmentID     0
Gender            0
ScheduledDay      0
AppointmentDay    0
Age               0
Neighbourhood     0
Scholarship       0
Hipertension      0
Diabetes          0
Alcoholism        0
Handcap           0
SMS_received      0
No-show           0
dtype: int64


In [14]:


#Feature Extraction
#Extract the following features:
#Gender
#Age
#Scholarship
#Hipertension
#Diabetes
#Alcoholism
#Handcap
#SMS_received
#Note: you may see incorrect spelling. It is related to the dataset, for example:
#Hipertension=Hypertension
#Handcap=Handicap
df_processed = df
df_processed = df_processed.drop(['PatientId','AppointmentID','ScheduledDay','Neighbourhood', 'AppointmentDay'],axis = 1)


column_names = ['Gender', 'Age', 'Scholarship',
       'Hypertension', 'Diabetes', 'Alcoholism', 'Handicap', 'SMS_received','No-show']
df_processed.columns = column_names




In [15]:


#Preprocessing
#Perform any needed pre-processing of the chosen features including:
#Scaling

df_processed['Age']=StandardScaler().fit_transform(df_processed['Age'].values.reshape(-1, 1))



#Encoding


df_processed[['No-show']] = df_processed[['No-show']].apply(lambda col: LabelEncoder().fit_transform(col))

encoded_Column = OneHotEncoder().fit_transform(df_processed['Gender'].values.reshape(-1, 1))
data_other_cols = df_processed.drop(columns=['Gender'])
df_processed = pd.concat([pd.DataFrame(encoded_Column.toarray(), columns = ['Gender_M','Gender_F']), data_other_cols], axis=1)
df_processed


,Gender_M,Gender_F,Age,Scholarship,Hypertension,Diabetes,Alcoholism,Handicap,SMS_received,No-show
0,1.0,0.0,1.077932,0,1,0,0,0,0,0
1,0.0,1.0,0.818306,0,0,0,0,0,0,0
2,1.0,0.0,1.077932,0,0,0,0,0,0,0
3,1.0,0.0,-1.258708,0,0,0,0,0,0,0
4,1.0,0.0,0.818306,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
110522,1.0,0.0,0.818306,0,0,0,0,0,1,0
110523,1.0,0.0,0.601950,0,0,0,0,0,1,0
110524,1.0,0.0,-0.696184,0,0,0,0,0,1,0
110525,1.0,0.0,0.039425,0,0,0,0,0,1,0


In [16]:


#Hint: use only the preprocessing steps you believe are useful.
#Splitting the Data
#Split your data as follows:
#80% training set
#10% validation set
#10% test set

y = df_processed["No-show"].values
X = df_processed[['Gender_M','Gender_F', 'Age', 'Scholarship',
       'Hypertension', 'Diabetes', 'Alcoholism', 'Handicap', 'SMS_received']].values


X_train, X_remaining, y_train, y_remaining = train_test_split(X, y, test_size=0.20, random_state=0)

X_val, X_test, y_val, y_test = train_test_split(X_remaining, y_remaining, test_size=0.5, random_state=1)


# decision-tree classifier

In [17]:

#Training Tree-based Classifiers
#Use a decision-tree classifier model to train your data.
#Choose the best criterion for the decision tree algorithm by trying different values and validating performance on the validation set.
#Note: choosing the best criterion is an example of hyper-parameter tuning.

print("decision-tree!!!!!!!!!!!!!!!!")
accuracy_score = []

criterion_options = ['entropy','gini','log_loss']
for i in range(len(criterion_options)):
    model_temp = DecisionTreeClassifier(criterion=criterion_options[i],random_state=0)
    model_temp.fit(X_train,y_train)
    accuracy_score.append(model_temp.score(X_test, y_test))
print("all accuracy score :",accuracy_score)    


model = DecisionTreeClassifier(criterion='entropy',random_state=0)
model.fit(X_train,y_train)
y_pred= model.predict(X_test)


#Classification Metrics
#Print the accuracy score of your final classifier.
final_accuracy_score = model.score(X_test, y_test)
print("final accuracy score :",final_accuracy_score)

#Print the confusion matrix.
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))



decision-tree!!!!!!!!!!!!!!!!
all accuracy score : [0.7977924545372297, 0.797701981362526, 0.7977924545372297]
final accuracy score : 0.7977924545372297
Confusion Matrix:
[[8787   65]
 [2170   31]]


# Random Forest

In [18]:
#Random Forest
#Repeat step 6.
#Increase/decrease the number of estimators in random forest and comment on the difference of the classification metrics.


accuracy_score2 = []

print("Random Forest!!!!!!!!!!!!!!!!")


for i in range(1,4):
    model_temp2 = RandomForestClassifier(criterion='entropy',random_state=0,n_estimators = i*100)
    model_temp2.fit(X_train,y_train)
    accuracy_score2.append(model_temp2.score(X_test, y_test))
print("all accuracy score :",accuracy_score2)   




model2 = RandomForestClassifier(criterion= 'entropy',random_state=0, n_estimators = 50)
model2.fit(X_train,y_train)
y_pred2= model2.predict(X_test)
#Print the accuracy score of your final classifier.
final_accuracy_score = model2.score(X_test, y_test)
print("final accuracy score :",final_accuracy_score)

#Print the confusion matrix.
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred2))

df_processed


Random Forest!!!!!!!!!!!!!!!!
all accuracy score : [0.7973400886637112, 0.7969781959648964, 0.7970686691396001]
final accuracy score : 0.7970686691396001
Confusion Matrix:
[[8775   77]
 [2166   35]]


,Gender_M,Gender_F,Age,Scholarship,Hypertension,Diabetes,Alcoholism,Handicap,SMS_received,No-show
0,1.0,0.0,1.077932,0,1,0,0,0,0,0
1,0.0,1.0,0.818306,0,0,0,0,0,0,0
2,1.0,0.0,1.077932,0,0,0,0,0,0,0
3,1.0,0.0,-1.258708,0,0,0,0,0,0,0
4,1.0,0.0,0.818306,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
110522,1.0,0.0,0.818306,0,0,0,0,0,1,0
110523,1.0,0.0,0.601950,0,0,0,0,0,1,0
110524,1.0,0.0,-0.696184,0,0,0,0,0,1,0
110525,1.0,0.0,0.039425,0,0,0,0,0,1,0
